In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from keras.applications.vgg16 import preprocess_input, decode_predictions
from glob import glob
import matplotlib.pyplot as plt
import tensorflow as tf
IMAGE_SIZE = [224, 224]
train_path = 'C:/PROJECTS/Workspace/AI/facemask/data/train1'
valid_path = 'C:/PROJECTS/Workspace/AI/facemask/data/test1'

# Root directory of the project
ROOT_DIR1 = os.path.abspath("../")
print ("ROOT DIR 1",ROOT_DIR1)
train_path = os.path.join(ROOT_DIR1, "facemask", "data", "train1")
valid_path = os.path.join(ROOT_DIR1, "facemask", "data", "test1")

print ("Train Path", train_path)
print ("Test Path", valid_path)



vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in vgg.layers:
    layer.trainable = False
  

folders = glob(train_path)
x = Flatten()(vgg.output)
prediction = Dense(len(folders), activation='sigmoid')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()
model.compile(
  loss= 'binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory(valid_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'binary')

r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)
model.save('C:/PROJECTS/Workspace/AI/facemask/maskmodel.h5')


In [ ]:
from PIL import Image
from keras.applications.vgg19 import preprocess_input
import base64
from io import BytesIO
import json
import random
import cv2
from keras.models import load_model
import numpy as np
from keras.preprocessing import image
model = load_model('C:/PROJECTS/Workspace/AI/facemask/maskmodel.h5')
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_extractor(img):
    faces=face_cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=20)
    
    if faces is ():
        return None
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face
video_capture = cv2.VideoCapture(0)
while True:
    _, frame = video_capture.read()
    face=face_extractor(frame)
    if type(face) is np.ndarray:
        face = cv2.resize(face, (224, 224))
        im = Image.fromarray(face, 'RGB')
        img_array = np.array(im)
        img_array = np.expand_dims(img_array, axis=0)
        pred = model.predict(img_array)
        #print(pred)                   
        name="None matching"
        
        if(pred[0][0]==0.):
            name='Mask Detected: Permitted'
            cv2.putText(frame,name, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
        else:
            name='No Mask: Not Permitted'
            cv2.putText(frame,name, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (139,0,0), 2)              
    else:
        cv2.putText(frame,"No face found", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)
    cv2.imshow('Video', frame)
    if cv2.waitKey(2)==27:
                break    
video_capture.release()
cv2.destroyAllWindows()

       



In [ ]:
"""
from PIL import Image, ImageDraw

#import face_recognition

# Load the jpg file into a numpy array
image = face_recognition.load_image_file("two_people.jpg")

# Find all facial features in all the faces in the image
face_landmarks_list = face_recognition.face_landmarks(image)

print("I found {} face(s) in this photograph.".format(len(face_landmarks_list)))

# Create a PIL imagedraw object so we can draw on the picture
pil_image = Image.fromarray(image)
d = ImageDraw.Draw(pil_image)

for face_landmarks in face_landmarks_list:

    # Print the location of each facial feature in this image
    for facial_feature in face_landmarks.keys():
        print("The {} in this face has the following points: {}".format(facial_feature, face_landmarks[facial_feature]))

    # Let's trace out each facial feature in the image with a line!
    for facial_feature in face_landmarks.keys():
        d.line(face_landmarks[facial_feature], width=5)

# Show the picture
pil_image.show()
"""

In [ ]:
import numpy as np
import cv2
import sys
from glob import glob
import itertools as it



face1 = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
video_capture = cv2.VideoCapture(0)
overlay = cv2.imread("gDean.png", -1)

def detect(img, cascade):
    rects = cascade.detectMultiScale(img, scaleFactor=1.3, minNeighbors=4, minSize=(30, 30), flags = cv2.CASCADE_SCALE_IMAGE)
    if len(rects) == 0:
        return []
    rects[:,2:] += rects[:,:2]
    return rects

def draw_overlay(img, rect):
    x1, y1, x2, y2 = rect
    y=y2-y1 + 40
    x=x2-x1 + 40
    small = cv2.resize(overlay, (x, y))

    x_offset = x1 - 10
    y_offset = y1 - 10

    for c in range(0,3):
        img[y_offset:y_offset + small.shape[0], x_offset:x_offset+ small.shape[1], c] = small[:,:,c] * (small[:,:,3]/255.0) + img[y_offset:y_offset+small.shape[0], x_offset:x_offset+small.shape[1], c] * (1.0 - small[:,:,3]/255.0)

def draw_rects(img, rects, color):
    for x1, y1, x2, y2 in rects:
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)

ret = True
while ret:
    # Capture frame-by-frame
    print("video capture read -1",video_capture.read(-1))
    ret, frame = video_capture.read(-1)

    if ret == True:
      gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
      gray = cv2.equalizeHist(gray)

      found = detect(gray, face1)

      if len(found) > 0:
          #for rect in found:
          #    draw_overlay(frame, rect)

          draw_rects(frame, found, (0, 255, 0))

      # Display the resulting frame
      cv2.imshow('laugh', frame)
    if cv2.waitKey(2)==27:
                break

# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()

In [5]:



import cv2
import numpy as np

# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

# Initialize Webcam
cap = cv2.VideoCapture(0)
count = 0

# Collect 100 samples of your face from webcam input
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = 'C:/PROJECTS/Workspace/AI/facemask/samples/face' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not f

Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not f

In [6]:
print(file_name_path)

C:/PROJECTS/Workspace/AI/facemask/samples/face100.jpg


In [7]:


import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
print(cv2.__version__)
# Get the training data we previously made
data_path = 'C:/PROJECTS/Workspace/AI/facemask/samples/'
# a=listdir('d:/faces')
# print(a)
# """
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)
# 
# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)
model=cv2.face_LBPHFaceRecognizer.create()

# Initialize facial recognizer
# model = cv2.face_LBPHFaceRecognizer.create()
# model=cv2.f
# NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()

# Let's train our model 
model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessefully")



4.3.0
Model trained sucessefully


In [8]:


import cv2
import numpy as np
import webbrowser
import os

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


# Open Webcam
cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Pass face to prediction model
        # "results" comprises of a tuple containing the label and the confidence value
        results = model.predict(face)
        print(results)
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident'
            
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        
        if confidence > 85:
            #os.system("usermod -U  harry")
            cv2.putText(image, "Hey Arun", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
                       
        else:
            cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )

    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()   

(87, 35.47795014070679)
(86, 36.44272219360458)
(87, 34.12927701323201)
(87, 36.66699501482926)
(82, 35.27141165356075)
(86, 33.78627938669447)
(82, 34.12409812228648)
(85, 32.73530016456094)
(82, 33.81737876869914)
(82, 32.1613574818084)
(84, 32.479279407473406)
(70, 32.71609664989976)
(70, 32.509173154689115)
(70, 33.493144183494145)
(66, 32.929115413853424)
(66, 32.536538730371035)
(66, 33.056830843059174)
(65, 30.667667656083506)
(66, 32.213944662505504)
(65, 31.683458474757778)
(65, 32.62649752418616)
(65, 31.219722149871764)
(65, 36.39470164266496)
(61, 34.938257143373924)
(65, 32.66247520531748)
(61, 31.457326916509967)
(61, 33.110503612244706)
(71, 32.915990774199194)
(61, 32.60253930173465)
(65, 32.67503462162281)
(54, 32.943570718771525)
(54, 32.29204885380649)
(65, 32.02733435227434)
(65, 35.55685408757763)
(71, 31.87541131594587)
(65, 32.18416482077769)
(61, 31.425279652012502)
(65, 31.9833733938857)
(65, 31.5620854252746)
(65, 31.724544645524453)
(71, 31.96356999298545)
(6

(61, 30.32350000103479)
(61, 31.551201283345783)
(61, 29.22425146254839)
(61, 30.749882997684843)
(65, 32.5394008307484)
(48, 36.24920321549411)
(48, 36.51711007129968)
(70, 32.32441909076498)
(77, 31.782841094379627)
(70, 29.394624801070595)
(77, 30.076023027337847)
(70, 28.24850657951511)
(72, 29.689507508164212)
(70, 29.83603878863398)
(70, 28.77212022992841)
(70, 29.07851038339493)
(70, 28.611459603588514)
(70, 29.021786984398265)
(70, 31.083975700266528)
(70, 29.009676872043716)
(70, 28.75618610220056)
(70, 29.161458641031523)
(70, 30.279561600810027)
(70, 30.447831765062674)
(70, 30.263957021649386)
(72, 28.779477188485295)
(70, 28.879922109282006)
(70, 28.96904171947458)
(70, 29.789802876981636)
(70, 29.74350920019656)
(70, 30.31705534654612)
(70, 30.10734065880784)
(70, 29.706758600014062)
(70, 29.851142230424987)
(65, 31.363072055916486)
(65, 30.819074920605367)
(70, 30.18351819909095)
(70, 29.9351560141968)
(70, 30.31064764944501)
(65, 29.716719191264175)
(70, 29.279733089279

(62, 28.885412557964685)
(72, 28.113648841697774)
(70, 27.119962025122987)
(72, 27.314460010709915)
(72, 27.06195364489552)
(72, 26.725857558826824)
(73, 26.945508613024348)
(70, 27.956174227665578)
(70, 27.87861385693571)
(61, 28.514257557158942)
(61, 29.285539971903955)
(61, 29.487072812557297)
(72, 28.606317925735222)
(65, 28.438489685946063)
(65, 27.73802915438405)
(65, 28.466201991793767)
(61, 28.477365586320108)
(72, 29.198363075063575)
(61, 28.782373780516)
(61, 31.081654874563707)
(65, 32.80271877791601)
(61, 30.706616816322715)
(61, 31.681826800958138)
(70, 30.43185743905097)
(6, 32.44073207506792)
(6, 32.60138107511541)
(83, 31.895413967054342)
(82, 33.52538431086567)
(83, 31.788527205970176)
(72, 33.28005921599321)
(81, 32.54726012295523)
(82, 32.0526232682527)
(82, 31.871921496767655)
(80, 32.00486624580477)
(82, 31.604428617151207)
(61, 34.39579612971829)
(48, 37.32589435242964)
(48, 37.13060081343087)
(61, 37.390709536459624)
(61, 34.881917219832)
(72, 35.32918588430241)
